In [44]:
traindf = pd.read_csv('./data/train_file_prod3x3StopFake.csv',sep=';',index_col=0,header=None)


In [45]:
test = pd.read_csv('./data/test_file_prod3x3StopFake.csv',sep=';',index_col=0,header=None)


In [46]:
classes = pd.read_csv('./data/PRClassesNoBlankSpaces.csv',sep=';',index_col=0,header=None)

In [47]:
classes.head()

,1
0,
1,Throws-Logging-UI-UserPreferences-Utils-DataSt...
7,IO-Utils-Throws-Logging-Network-UI-UserPrefere...
10,DataStructure-Utils
11,Utils-TextUtility-IO-DataStructure
15,Concurrency-Logging-DataStructure


In [48]:
classes.shape

(1639, 1)

In [49]:
traindf.shape

(741, 21)

In [50]:
test.shape

(404, 21)

In [42]:
import numpy as np
import pandas as pd
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
  

def train():
    train = pd.read_csv('./data/train_file_prod3x3StopFake.csv',sep=';',index_col=0,header=None)
    test = pd.read_csv('./data/test_file_prod3x3StopFake.csv',sep=';',index_col=0,header=None)
    classes = pd.read_csv('./data/PRClassesNoBlankSpaces.csv',sep=';',index_col=0,header=None)
    train = train.fillna(0)
    test = test.fillna(0)
    
    ids = test.index

    cls = [data[0].split('-') for data in classes.values]
    mlb = MultiLabelBinarizer()
    cls = mlb.fit_transform(cls)
    classes = pd.DataFrame(cls,index=classes.index)


    X_train = train.values  
    X_test = test.values
    y_train = classes.loc[train.index].values
    y_test = classes.loc[test.index].values

    #clf = BinaryRelevance(classifier=SVC(),require_dense=[False, True])
    clf = BinaryRelevance(classifier=DecisionTreeClassifier(), require_dense = [False, True])
    clf.fit(X_train,y_train)
    
    probability = clf.predict_proba(X_test).toarray()
    predict = clf.predict(X_test).toarray()

    predict = pd.DataFrame(predict,index=ids, columns=mlb.classes_) # with header
    probability = pd.DataFrame(probability,index=ids, columns=mlb.classes_) # with header

    #predict = pd.DataFrame(predict,index=ids)
    #probability = pd.DataFrame(probability,index=ids)

    predict.to_csv('./data/predict_file_3x3Stop.csv')
    probability.to_csv('./data/probability_file_3x3Stop.csv')
    
    return classes, clf, y_train, y_test, X_test
    



In [43]:
classes, clf, y_train, y_test, X_test = train()

In [7]:
classes1 = pd.read_csv('./data/PRClassesNoBlankSpaces.csv',sep=';',index_col=0,header=None)

In [8]:
classes1.head()


,1
0,
1,Throws-Logging-UI-UserPreferences-Utils-DataSt...
7,IO-Utils-Throws-Logging-Network-UI-UserPrefere...
10,DataStructure-Utils
11,Utils-TextUtility-IO-DataStructure
15,Concurrency-Logging-DataStructure


In [9]:
classes1.iloc[:1].values


array([['Throws-Logging-UI-UserPreferences-Utils-DataStructure-IO-Network']],
      dtype=object)

In [10]:
y_pred = clf.predict(X_test).toarray()

NameError: name 'clf' is not defined

In [ ]:

from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test).toarray()
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=classes1.iloc[:1].values, yticklabels=classes1.iloc[:1].values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()



In [ ]:
skm.multilabel_confusion_matrix(y_true, y_pred)